In [ ]:
#import tweepy # module for downloading tweets
#from tweepy import OAuthHandler,AppAuthHandler, Cursor
import pandas as pd
import sqlite3
import os
import json
import sys
import numpy as np

# tweepy basic setup
import tweepy
from tweepy import OAuthHandler

import ast
import re

import config

In [ ]:
# Uncomment and enter info from twitter API or create config.py file
#consumer_key = 
#consumer_secret = 
#access_token = 
#access_secret =
 
auth = OAuthHandler(config.consumer_key, config.consumer_secret)
auth.set_access_token(config.access_token, config.access_secret)

# setting wait_on_rate_limit prevents overuse
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
# need to start with a file of possible trolls (from a blocklist)
user_df = pd.read_csv(open('trollsJune20full.csv','rU'), encoding='utf-8', engine='python')

# or file of possible humans (general users, on at least one list)


# add a column 'downloaded' and initialize with False
user_df['downloaded']=False

In [ ]:
def get_tweets(user_df,block,filename,number=200)
    '''
    This function grabs all of the tweets for the users in user_df. This should be a dataframe of 
    all blocked users (block=1) or all not blocked (block=0). Enter filename for database (but can be new SQL file
    so long as it is in format specified below. Default number of tweets to grab is 200.)
    '''

    # Opening a connection to the sqlite3 database
    # create a filename database (e.g. tweets.db) if there is not one right now
    # else connect to the current database

    if not os.path.exists('filename'):
        conn = sqlite3.connect('filename')
        c = conn.cursor()
        # creating a table if it does not exist.
        c.execute('''CREATE TABLE tweets (tweet TEXT, tweet_id INTEGER, screen_name TEXT, mentions TEXT, created_at TEXT, troll INTEGER)''')
    else:
        conn = sqlite3.connect('filename')
        c = conn.cursor()

    #initialize a list to hold all the tweepy Tweets
    alltweets = []
    new_tweets = []

    # look in user_df at each row
    for index, row in user_df.iterrows():
        if row['downloaded']==False:
            print('Getting {x}'.format(x=row['screen_name']))

        #make request for most recent tweets
        try:
            new_tweets = api.user_timeline(screen_name = row['screen_name'],count=number)
        except tweepy.TweepError as e:
            print('Error ')
            print(e.api_code)
            pass

        # write these to the sql database
        for tweet in new_tweets: #Saving tweets to SQL
            tweet._json #First convert tweets from string to json object.

            # collect my variables of interest
            text = tweet.text 
            tweet_id = tweet.id
            screen_name = tweet.user.screen_name
            mentions = re.sub(r'[^\x00-\x7f]',r'', str(tweet.entities))
            mentions = str(ast.literal_eval(mentions)['user_mentions'])
            created_at = tweet.created_at
            troll = block 

            c.execute("INSERT INTO tweets VALUES (?,?,?,?,?,?)",(text,tweet_id,screen_name,mentions,created_at,troll))
            conn.commit()  

            user_df.loc[index, 'downloaded']=True

    conn.close() #Close connection.
